In [1]:
import h5py
import os
from tqdm import tqdm
from os import listdir
from os.path import isfile, join
import numpy as np

In [2]:
def lowe_ratio_nn(kp1_desc, kp2_desc, r=0.95):
    # Calculate l2 distance for all possible pairs. The resulting matrix should be of size n_keypoints_1 x n_keypoints_2
    dist = ((kp1_desc[:, None, :] - kp2_desc[None, :, :]) ** 2).sum(axis=-1)

    # Find closest matching descriptor from kp2_desc. The resulting vector should be of size n_keypoints_1
    nn_idx1 = np.argsort(dist, axis=1)[:, 0]
    ratio_test_mask1 = dist[range(dist.shape[0]), nn_idx1] / np.sort(dist, axis=1)[:, 1] < r

    # Find closest matching descriptor from kp1_desc. The resulting vector should be of size n_keypoints_2
    nn_idx2 = np.argsort(dist, axis=0)[0, :]
    ratio_test_mask2 = dist[nn_idx2, range(dist.shape[1])] / np.sort(dist, axis=0)[1, :] < r

    # Find mutual matches here
    mutual_mask = np.zeros(len(nn_idx1))
    for i, m in enumerate(nn_idx1):
        if nn_idx2[m] == i:
            mutual_mask[i] = ratio_test_mask2[m]
    # Combine mutual_mask and ratio test masks here from the perspective of nn_idx1
    match_mask = ratio_test_mask1 * mutual_mask
    nn_idx1[match_mask.astype(int) == 0] = -1
    return nn_idx1

In [7]:
mypath = "../all_data_superglue/rgb_small/"
img_names = sorted([f for f in listdir(mypath) if isfile(join(mypath, f))])
    
h5_path = "../all_data_superglue/rgb_artifacts/"

keypoint_f = h5py.File(os.path.join(h5_path, 'keypoints.h5'), 'r')
keypoints = list(keypoint_f.keys())

descriptors_f = h5py.File(os.path.join(h5_path, 'descriptors.h5'), 'r')
descriptors = list(descriptors_f.keys())

In [4]:
for i in tqdm(range(len(descriptors) - 1)):
    name1 = img_names[i][:-4]
    name2 = img_names[i+1][:-4]
    desc1 = descriptors_f[descriptors[i]][()]
    desc2 = descriptors_f[descriptors[i + 1]][()]
    kp1 = keypoint_f[keypoints[i]][()]
    kp2 = keypoint_f[keypoints[i + 1]][()]
    match = lowe_ratio_nn(desc1, desc2)
    np.savez(f"./disk_lowe_rgb/{name1}_{name2}_matches.npz", {'keypoints0': kp1, 'keypoints1': kp2, 'matches': match})

100%|███████████████████████████████████████████| 19/19 [00:06<00:00,  3.02it/s]


In [8]:
glue_path = "../all_data_superglue/rgb_glue_artifacts/"
glue_names = img_names = sorted([f for f in listdir(glue_path) if isfile(join(glue_path, f))])

In [9]:
for name in tqdm(glue_names):
    info = dict(np.load(glue_path + name))
    match = lowe_ratio_nn(info['descriptors0'].T, info['descriptors1'].T)
    np.savez('./glue_lowe_rgb/' + name, {'keypoints0': info['keypoints0'], 'keypoints1': info['keypoints1'], 'matches': match})

100%|███████████████████████████████████████████| 19/19 [00:01<00:00, 18.89it/s]


In [16]:
info['descriptors0'].shape, info['keypoints0'].shape, info['scores0'].shape

((256, 367), (367, 2), (367,))